In [ ]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [ ]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [21]:
feedback_bits = 256
ncodebooks=256
ncentroids=16
train_sam_num = 3000 # 训练集样本数
split = 8 # A的列、B的行分割成split份

In [ ]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")

dir_train_split = os.path.join(dir_train, 'train', 'f'+str(feedback_bits), 'split'+str(split))
# print(dir_train_split)
dir_test_split = os.path.join(dir_train, 'test', 'f'+str(feedback_bits), 'split'+str(split))


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



切分训练集

In [ ]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_train_split_path_list = []
for i in range(split):
    data_to_fc_train_split_path_list.append(os.path.join(dir_train_split, 'data_to_fc_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_train_split_path_list[i], data_to_fc_train[np.ix_(range(data_to_fc_train.shape[0]), range(i*split_size,(i+1)*split_size))])

In [ ]:
weight_train = np.load(os.path.join(dir_train, weightpath))
print(weight_train.shape)
weight_train_split_path_list = []
for i in range(split):
    weight_train_split_path_list.append(os.path.join(dir_train_split, 'weight_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(weight_train_split_path_list[i], weight_train[np.ix_(range(i*split_size,(i+1)*split_size), range(weight_train.shape[1]))])

In [ ]:
# 输入与weight相乘
y_train_split_path_list = []
for i in range(split):
    y_train_split_path_list.append(os.path.join(dir_train_split, 'y_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_train_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_train_split_path_list[i], np.matmul(xx, ww))

切分测试集

In [18]:
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
# split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_test_split_path_list = []
for i in range(split):
    data_to_fc_test_split_path_list.append(os.path.join(dir_test_split, 'data_to_fc_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_test_split_path_list[i], data_to_fc_test[np.ix_(range(data_to_fc_test.shape[0]), range(i*split_size,(i+1)*split_size))])

(32000, 2048)


In [19]:
# 输入与weight相乘
y_test_split_path_list = []
for i in range(split):
    y_test_split_path_list.append(os.path.join(dir_test_split, 'y_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_test_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_test_split_path_list[i], np.matmul(xx, ww))

In [22]:
est_list = []
for i in range(split):
    dir_est, X_path = os.path.split(data_to_fc_train_split_path_list[i])
    dir_est, W_path = os.path.split(weight_train_split_path_list[i])
    dir_est, Y_path = os.path.split(y_train_split_path_list[i])
    est3 = mm.estFactory(X_path=X_path, W_path=W_path, Y_path=Y_path, dir= dir_est, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])
    est_list.append(est3)


running method:  Mithral
learn_multisplits(): initial loss:    0.3427701936947415
learn_multisplits(): returning loss:  0.0031355529281427152
learn_multisplits(): initial loss:    0.068633385400934
learn_multisplits(): returning loss:  0.0008765891871007625
learn_multisplits(): initial loss:    0.15058208930940586
learn_multisplits(): returning loss:  0.002012650791584747
learn_multisplits(): initial loss:    0.07807186071212284
learn_multisplits(): returning loss:  0.0009726802309160121
learn_multisplits(): initial loss:    0.12152430667246865
learn_multisplits(): returning loss:  0.001777797606337117
learn_multisplits(): initial loss:    0.045222607148025416
learn_multisplits(): returning loss:  0.0008673737502249423
learn_multisplits(): initial loss:    0.1178508801997177
learn_multisplits(): returning loss:  0.0021772048021375667
learn_multisplits(): initial loss:    0.45442995069966663
learn_multisplits(): returning loss:  0.01717581848060945
learn_multisplits(): initial loss:    

In [23]:
print(est_list)

[<vq_amm.MithralMatmul object at 0x7f3063828460>, <vq_amm.MithralMatmul object at 0x7f302bd82310>, <vq_amm.MithralMatmul object at 0x7f30639f0760>, <vq_amm.MithralMatmul object at 0x7f30253cd070>, <vq_amm.MithralMatmul object at 0x7f3062ffa130>, <vq_amm.MithralMatmul object at 0x7f302540d130>, <vq_amm.MithralMatmul object at 0x7f30252022b0>, <vq_amm.MithralMatmul object at 0x7f305cb670d0>]


In [24]:
y_split_out_matmul_list = []
for i in range(split):
    x_test = np.load(data_to_fc_test_split_path_list[i])
    w_test = np.load(weight_train_split_path_list[i])
    y_split_out_matmul = mm.eval_matmul(est_list[i], x_test, w_test)
    y_split_out_matmul_list.append(y_split_out_matmul)

In [25]:
# split 的矩阵结果合成
y_out_matmul = y_split_out_matmul_list[0]
if split>1:
    for i in range(1, split):
        y_out_matmul += y_split_out_matmul_list[i]

In [26]:
bias = np.load(dir_train+'/'+biaspath)
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [27]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_split%i_fb%i_cb%i_ct%i.npy' % (split, feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.23177403  0.02442732 -0.6081972  ...  0.06121831  0.11843655
  -0.017315  ]
 [-0.15741006 -0.20257856 -0.2989996  ... -0.50629634  0.0205892
  -0.09167899]
 [ 0.3161712  -0.00296994 -0.27551624 ... -0.62371314  0.22019781
   0.15098247]
 ...
 [ 0.44141576  0.11836077 -0.42424423 ... -0.2323238  -0.09291375
   0.01399616]
 [ 0.17918485 -0.15561183 -0.15809941 ...  0.1316684  -0.08117206
   0.03356563]
 [-0.41572705  0.14967194 -0.47903872 ... -0.48672697 -0.09682766
   0.13532689]]
(32000, 256)
